# Exploring the Polygon ID JS-SDK, step by step (in TypeScript)

Mostly based on tutorial at https://0xpolygonid.github.io/js-sdk-tutorials/ and its code in repo: https://github.com/iden3/polygonid-js-sdk-examples.

General Iden3 tutorial: https://0xpolygonid.github.io/tutorials/, with code at https://github.com/0xPolygonID/tutorial-examples.  
See fork / branch at https://github.com/nedgar/tutorial-examples/tree/no-publish-state for a simplified / clarified version.

API reference: https://0xpolygonid.github.io/js-sdk-tutorials/docs/api/

To set up Jupyter with TypeScript kernel, see https://github.com/yunabe/tslab.

In [1]:
// use at least node v16
process.version

v18.15.0


In [2]:
import _ from "lodash"
import path from "path"

## Configuration

In [3]:
import config from "./src/config"

const { rhsUrl, rpcUrl, contractAddress: stateContractAddress, circuitsFolder } = {
    ...config,
    circuitsFolder: 'circuits' // override for notebook
}

In [4]:
// Revocation Hash Service (RHS)
rhsUrl

https://rhs-staging.polygonid.me


In [11]:
// JSON RPC URL for blockchain network (API KEY hidden)
if (rpcUrl.includes("YOUR_ALCHEMY_API_KEY")) { throw "missing API key" }
rpcUrl.replace(/[A-Za-z0-9]+$/, "*****")

https://polygon-mumbai.g.alchemy.com/v2/*****


In [12]:
stateContractAddress

0x134B1BE34911E39A8397ec6289782989729807a4


## Storage / State

In [13]:
import { Identity, IdentityStorage, InMemoryDataSource, Profile } from "@0xpolygonid/js-sdk"

// storage for identities and their profiles (secondary identities derived from nonce)
const identityStorage = new IdentityStorage(
    new InMemoryDataSource<Identity>(),
    new InMemoryDataSource<Profile>()
)

In [14]:
import { CredentialStorage, W3CCredential } from "@0xpolygonid/js-sdk"

// storage for W3C Verifiable Credentials (VCs)
const credentialStorage = new CredentialStorage(new InMemoryDataSource<W3CCredential>())

In [15]:
import { InMemoryMerkleTreeStorage } from "@0xpolygonid/js-sdk"

// storage for Merkle trees, e.g. for an identity's claims tree, revocations tree, prior roots tree
const mtDepth = 40
const mtStorage = new InMemoryMerkleTreeStorage(mtDepth)

In [16]:
2 ** mtDepth

1099511627776


In [17]:
import { EthConnectionConfig, EthStateStorage, defaultEthConnectionConfig } from "@0xpolygonid/js-sdk"

// JSON RPC connection config
const ethConnectionConfig: EthConnectionConfig = {
    ...defaultEthConnectionConfig,
    contractAddress: stateContractAddress,
    url: rpcUrl,
}

// on-chain storage for ID states and corresponding MT roots for:
// - claims tree
// - revocations tree
// - (previous) roots tree
const stateStorage = new EthStateStorage(ethConnectionConfig)

In [18]:
import { IDataStorage } from "@0xpolygonid/js-sdk"

// the various data sources, collected in one object
const dataStorage: IDataStorage = {
    credential: credentialStorage,
    identity: identityStorage,
    mt: mtStorage,
    states: stateStorage,
}

In [19]:
import { CircuitData, CircuitId, CircuitStorage, ICircuitStorage, FSKeyLoader } from "@0xpolygonid/js-sdk"

const circuitStorage = new CircuitStorage(new InMemoryDataSource<CircuitData>());

const circuitIds = [
    CircuitId.AuthV2, 
    CircuitId.AtomicQuerySigV2, 
    CircuitId.StateTransition, 
    // CircuitId.AtomicQueryMTPV2
]
for (let circuitId of circuitIds) {
    console.log("Loading circuit:", circuitId);

    const loader = new FSKeyLoader(path.join(circuitsFolder, circuitId));
    await circuitStorage.saveCircuitData(circuitId, {
        circuitId: circuitId,
        wasm: await loader.load("circuit.wasm"),
        provingKey: await loader.load("circuit_final.zkey"),
        verificationKey: await loader.load("verification_key.json"),
    })
}

Loading circuit: authV2
Loading circuit: credentialAtomicQuerySigV2
Loading circuit: stateTransition


## KMS and Wallets

In [20]:
import { CredentialWallet } from "@0xpolygonid/js-sdk"

// credential wallet
const credentialWallet = new CredentialWallet(dataStorage)

In [21]:
import { BjjProvider, KmsKeyType, InMemoryPrivateKeyStore, KMS } from "@0xpolygonid/js-sdk"

// configure KMS with private keystore and BJJ key provider
const kms = new KMS();
const keyStore = new InMemoryPrivateKeyStore();
const bjjProvider = new BjjProvider(KmsKeyType.BabyJubJub, keyStore);
kms.registerKeyProvider(KmsKeyType.BabyJubJub, bjjProvider);

In [22]:
// TODO: ask Fireblocks re BJJ signing algorithm
// does Hashicorp Vault use support SGX

In [23]:
import { IdentityWallet } from "@0xpolygonid/js-sdk"

// identity wallet using KMS for private keys, and credential wallet to use when issuing credentials
const identityWallet = new IdentityWallet(kms, dataStorage, credentialWallet)

## Create identities and their auth credentials

In [24]:
import { core } from "@0xpolygonid/js-sdk"
// import * as core from "@iden3/js-iden3-core"  // need newer version with fix for https://github.com/iden3/js-iden3-core/pull/17

// create the issuer identity
const { did: issuerDID, credential: issuerAuthCredential } =
    await identityWallet.createIdentity(
      "http://wallet.com/", // this is url that will be a part of auth bjj credential identifier
      {
        // method: core.DidMethod.Iden3,
        method: core.DidMethod.PolygonId,
        blockchain: core.Blockchain.Polygon,
        networkId: core.NetworkId.Mumbai,
        rhsUrl: rhsUrl,
      }
    );

In [25]:
issuerDID.toString()

did:polygonid:polygon:mumbai:2qDrzvE2A813Fyg2sPE7JEA6f6yDsUJfzHXAhz9hZV


In [26]:
issuerDID.id.bigInt()

21734601890291053386314474656025329556666952536788676076983649605779329538n


In [27]:
issuerAuthCredential

W3CCredential {
  id: 'http://wallet.com/8f6bd919-f88f-4885-b57a-6f6bfedc3930',
  '@context': [
    'https://www.w3.org/2018/credentials/v1',
    'https://schema.iden3.io/core/jsonld/iden3proofs.jsonld',
    'https://schema.iden3.io/core/jsonld/auth.jsonld'
  ],
  type: [ 'VerifiableCredential', 'AuthBJJCredential' ],
  expirationDate: undefined,
  issuanceDate: '2023-03-22T19:28:27.648Z',
  credentialSubject: {
    x: '12231468819415572782146080053734790465814594053490310524360196993403827137604',
    y: '13441944608637263006807843726390751166827496335973566159515314843229151443541',
    type: 'AuthBJJCredential'
  },
  issuer: 'did:polygonid:polygon:mumbai:2qDrzvE2A813Fyg2sPE7JEA6f6yDsUJfzHXAhz9hZV',
  credentialSchema: {
    id: 'https://schema.iden3.io/core/json/auth.json',
    type: 'JsonSchemaValidator2018'
  },
  credentialStatus: {
    id: 'https://rhs-staging.polygonid.me',
    revocationNonce: 0,
    type: 'Iden3ReverseSparseMerkleTreeProof'
  },
  proof: [
    Iden3SparseMer

Suggestion: demo on-chain revocations.

Q: BJJ vs Secp256k1 and relationship to blockchain address / private key

A: ...

Q: What is the relationship between the DID and the user's public and private keys?

A: The Iden3 DID includes the base58 encoding of the identity's genesis state. This is not the public key (nor the private key, of course). 
In the genesis state, the claims Merkle tree has only the identity's auth claim, and the revocation and roots trees are empty. Since the ID state is the hash of (claims tree root, revocations tree root, prior roots tree root), the genesis state is effectively just a secondary hash of the auth claim. 

See below.

In [28]:
import { PublicKey } from "@iden3/js-crypto"

// the credential subject has the BJJ coordinates which combine to form the Iden3 core public key
const { x, y } = issuerAuthCredential.credentialSubject;
const pubKey = new PublicKey([BigInt(x), BigInt(y)])
pubKey.hex()

553a0e3308c707d5d137b0e2adbb2f816bca173b8323240b4cd91032fcdeb79d


In [29]:
// in the SDK, this is used as the KMS ID
const kmsId = (identityWallet as any).getKMSIdByAuthCredential(issuerAuthCredential);  // private method
kmsId

{
  type: 'BJJ',
  id: 'BJJ:553a0e3308c707d5d137b0e2adbb2f816bca173b8323240b4cd91032fcdeb79d'
}


In [30]:
// round trip: look up public key given KMS ID
(await kms.publicKey(kmsId)).hex()

553a0e3308c707d5d137b0e2adbb2f816bca173b8323240b4cd91032fcdeb79d


In [31]:
// The KMS can sign messages (using the private key), but the BJJ provider only supports signing ~253-bit values (passed as 32 bytes).
// Q: Why ~253 bits? The BJJ elliptic curve is on a finite field with prime number Q = 2^253 + ...), and all calculations are done MOD(Q). Inputs greater than Q are invalid.

let bytes = new Uint8Array(32).map((_, i) => i + 1)
console.log("message to sign:", bytes)
console.log("signature:", await kms.sign(kmsId, bytes))

message to sign: Uint8Array(32) [
   1,  2,  3,  4,  5,  6,  7,  8,  9,
  10, 11, 12, 13, 14, 15, 16, 17, 18,
  19, 20, 21, 22, 23, 24, 25, 26, 27,
  28, 29, 30, 31, 32
]
signature: Uint8Array(64) [
   99,  34,  40,  84, 222, 249, 246,  26,  42, 101, 201,
   72, 170, 218, 147,  87,  71,  25,  12, 238, 118, 125,
   47, 209, 190, 149,  31,   1, 116, 182,  91,   9, 247,
    8, 149, 181,  60,  50, 165, 220, 211, 127,  68, 142,
  115,  72, 230, 115,  83, 123, 178,  32, 126,  72,  59,
  238, 166,  71, 145, 190,  39,  41, 225,   3
]


In [32]:
// the auth credential's proof is an Iden3 Sparse MTP containing:
// - the Iden3 auth core claim (serialized)
// - issuer data with:
//   - the issuer's DID and ID state at time of issuance (the genesis state here, with no prior roots)
//   - the issuer's auth core claim (repeated, since it's self-issued)
//   - MTP of existence of claim in the issuer's claims tree (as a single leaf node, so no siblings)

issuerAuthCredential.proof[0]

Iden3SparseMerkleTreeProof {
  type: 'Iden3SparseMerkleTreeProof',
  mtp: Proof {
    existence: true,
    depth: 0,
    siblings: [],
    notEmpties: Uint8Array(30) [
      0, 0, 0, 0, 0, 0, 0, 0, 0,
      0, 0, 0, 0, 0, 0, 0, 0, 0,
      0, 0, 0, 0, 0, 0, 0, 0, 0,
      0, 0, 0
    ]
  },
  issuerData: IssuerData {
    id: 'did:polygonid:polygon:mumbai:2qDrzvE2A813Fyg2sPE7JEA6f6yDsUJfzHXAhz9hZV',
    state: {
      rootOfRoots: '0000000000000000000000000000000000000000000000000000000000000000',
      revocationTreeRoot: '0000000000000000000000000000000000000000000000000000000000000000',
      claimsTreeRoot: '8b0a842a3be8aa86c4c4851636940e3fe20f8edcc8f9b6bd460d6c0f29c31b2a',
      value: '2474a161301ebc4b83c7214ca7e64a708228dbcd78f6a50826dc681627ad3025'
    },
    authCoreClaim: 'cca3371a6cb1b715004407e325bd993c000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000442c565c84d08f4eb369aab37485d1db0e614cd25224205228c1e8c419c40a1b553a0e3308c707d

In [37]:
function describeClaim(claim: core.Claim) {
    // workaround for https://github.com/iden3/js-iden3-core/issues/16
    // function getId() {
    //     const pos = claim.getIdPosition()
    //     let bytes = null;
    //     switch (pos) {
    //         case core.IdPosition.Index:
    //             bytes = claim.index[1].bytes
    //             break
    //         case core.IdPosition.Value:
    //             bytes = claim.value[1].bytes
    //             break
    //     }
    //     return bytes && core.Id.fromBytes(bytes.slice(0, 31)).string()
    // }
    
    const anyClaim = claim as any
    return {
        schemaHash: claim.getSchemaHash().bigInt(),
        id: claim.getIdPosition() ? claim.getId().bigInt() : null,
        indexData: claim.index.slice(2).map(d => d.toBigInt()),
        valueData: claim.value.slice(2).map(d => d.toBigInt()),
        expirationDate: claim.getExpirationDate(),
        merklizedRoot: anyClaim.getMerklized() ? claim.getMerklizedRoot() : null,
        revocationNonce: claim.getRevocationNonce(),
        version: claim.getVersion(),
        flags: {
            subject: core.SubjectFlag[anyClaim.getSubject()],
            idPosition: core.IdPosition[claim.getIdPosition()],                                        
            expiration: anyClaim.getFlagExpiration(),
            merklized: !!anyClaim.getMerklized(),
            updatable: claim.getFlagUpdatable()
        }
    }
}

In [38]:
let claim = new core.Claim().fromHex(issuerAuthCredential.proof[0].coreClaim)
describeClaim(claim)

{
  schemaHash: 80551937543569765027552589160822318028n,
  id: null,
  indexData: [
    12231468819415572782146080053734790465814594053490310524360196993403827137604n,
    13441944608637263006807843726390751166827496335973566159515314843229151443541n
  ],
  valueData: [ 0n, 0n ],
  expirationDate: null,
  merklizedRoot: null,
  revocationNonce: 0n,
  version: 0,
  flags: {
    subject: 'Self',
    idPosition: 'None',
    expiration: false,
    merklized: false,
    updatable: false
  }
}


In [39]:
// auth claims use the standard Iden3 auth schema
core.SchemaHash.authSchemaHash.bigInt()

80551937543569765027552589160822318028n


In [40]:
// create the user (aka holder) identity
const { did: userDID, credential: userAuthCredential } = await identityWallet.createIdentity(
    "http://wallet.com/",
    {
      method: core.DidMethod.Iden3,
      blockchain: core.Blockchain.Polygon,
      networkId: core.NetworkId.Mumbai,
      rhsUrl: rhsUrl,
    }
)
userDID.toString()

did:iden3:polygon:mumbai:wuyewEJyZZYbXvHtoc844eQHUeK17CkeWDWj3gwfY


In [41]:
// // use DID from Polygon ID wallet instead of creating a new one (click person icon to view DID)
// const userDID = core.DID.parseFromId(core.Id.fromString("2qGKMXEQ53gNohAfxqYNPz6Kg1xYsBSN7ZQGG8tYMc"))
// userDID.toString()

## Issuer prompts user to sign in

In [42]:
import { AuthorizationRequestMessage, AuthorizationRequestMessageBody, PROTOCOL_CONSTANTS, ZeroKnowledgeProofRequest } from "@0xpolygonid/js-sdk"

const threadId = "d10c5633-9ea2-407d-90b0-cc9e91e40997"
const sessionId = 12345
let authRequest: AuthorizationRequestMessage = {
    id: threadId,
    thid: threadId,
    typ: PROTOCOL_CONSTANTS.MediaType.PlainMessage,
    type: PROTOCOL_CONSTANTS.PROTOCOL_MESSAGE_TYPE.AUTHORIZATION_REQUEST_MESSAGE_TYPE,
    from: issuerDID.toString(),
    body: {
      callbackUrl: `http://testcallback.com?sessionId=${sessionId}`,
      reason: "sign in",
      scope: [],
    }
}

JSON.stringify(authRequest, null, 2)

{
  "id": "d10c5633-9ea2-407d-90b0-cc9e91e40997",
  "thid": "d10c5633-9ea2-407d-90b0-cc9e91e40997",
  "typ": "application/iden3comm-plain-json",
  "type": "https://iden3-communication.io/authorization/1.0/request",
  "from": "did:polygonid:polygon:mumbai:2qDrzvE2A813Fyg2sPE7JEA6f6yDsUJfzHXAhz9hZV",
  "body": {
    "callbackUrl": "http://testcallback.com?sessionId=12345",
    "reason": "sign in",
    "scope": []
  }
}


In [43]:
import * as qrSvg from "qr-svg" 
import * as tslab from "tslab"

// present auth request as QR code so it can be scanned in Polygon ID app
let svg = qrSvg.QR(JSON.stringify(authRequest), "H");
tslab.display.html(`<div style="width: 300px; height: 300px;">${svg}</div>`)

## Set up Proof Service, Package Manager, and Auth Handler

Used by user's ID wallet to respond to auth requests, and verifier to verify them. Not used by issuer.

In [44]:
import { IProofService, ProofService } from "@0xpolygonid/js-sdk"

const proofService: IProofService = new ProofService(identityWallet, credentialWallet, circuitStorage, stateStorage)

In [45]:
import { proving } from "@iden3/js-jwz"
import { AuthDataPrepareFunc, DataPrepareHandlerFunc, IPackageManager, PackageManager, PlainPacker, ProvingParams, StateVerificationFunc, VerificationHandlerFunc, VerificationParams, ZKPPacker } from "@0xpolygonid/js-sdk"

async function initPackageManager(
  circuitData: CircuitData,
  prepareFn: AuthDataPrepareFunc,
  stateVerificationFn: StateVerificationFunc
): Promise<IPackageManager> {
    const mapKey = proving.provingMethodGroth16AuthV2Instance.methodAlg.toString();

    const verificationFn = new VerificationHandlerFunc(stateVerificationFn);
    const verificationParamMap: Map<string, VerificationParams> = new Map();
    verificationParamMap.set(mapKey, {
        key: circuitData.verificationKey,
        verificationFn,
    });

    const authInputsHandler = new DataPrepareHandlerFunc(prepareFn);
    const provingParamMap: Map<string, ProvingParams> = new Map();
    provingParamMap.set(mapKey, {
        dataPreparer: authInputsHandler,
        provingKey: circuitData.provingKey,
        wasm: circuitData.wasm,
    });

    const mgr: IPackageManager = new PackageManager();
    const packer = new ZKPPacker(provingParamMap, verificationParamMap);
    const plainPacker = new PlainPacker();
    mgr.registerPackers([packer, plainPacker]);

    return mgr;
}

In [46]:
const authCircuitData = await circuitStorage.loadCircuitData(CircuitId.AuthV2)
const pm = await initPackageManager(
    authCircuitData,
    proofService.generateAuthV2Inputs.bind(proofService),
    proofService.verifyState.bind(proofService)
)

In [47]:
authCircuitData

{
  circuitId: 'authV2',
  wasm: Uint8Array(4042224) [
      0,  97, 115, 109,   1,   0,   0,   0,   1, 102,  19,  96,
      2, 127, 127,   0,  96,   1, 127,   0,  96,   1, 127,   1,
    127,  96,   2, 127, 127,   1, 127,  96,   3, 127, 127, 127,
      1, 127,  96,   3, 127, 127, 127,   0,  96,   3, 127, 126,
    127,   0,  96,   2, 127, 126,   0,  96,   4, 127, 127, 127,
    127,   0,  96,   2, 126, 126,   1, 126,  96,   2, 127, 127,
      1, 126,  96,   0,   0,  96,   0,   1, 127,  96,   1, 127,
      0,  96,   1, 127,   1, 127,  96,   1, 126,   1, 127,  96,
      2, 127, 127,   0,
    ... 4042124 more items
  ],
  provingKey: Uint8Array(28795369) [
    122, 107, 101, 121,   1,   0,   0,   0,  10,   0,   0,   0,
      1,   0,   0,   0,   4,   0,   0,   0,   0,   0,   0,   0,
      1,   0,   0,   0,   2,   0,   0,   0, 148,   2,   0,   0,
      0,   0,   0,   0,  32,   0,   0,   0,  71, 253, 124, 216,
     22, 140,  32,  60, 141, 202, 113, 104, 145, 106, 129, 151,
     93,  88, 129, 1

In [48]:
import { AuthHandler } from "@0xpolygonid/js-sdk";

const authHandler = new AuthHandler(pm, proofService, credentialWallet)

## User responds to sign-in auth request

In [49]:
// scanning the QR code yields the auth request as bytes
let authRawRequest = new TextEncoder().encode(JSON.stringify(authRequest))
authRawRequest

Uint8Array(374) [
  123,  34, 105, 100, 34,  58, 34, 100,  49,  48,  99,  53,
   54,  51,  51,  45, 57, 101, 97,  50,  45,  52,  48,  55,
  100,  45,  57,  48, 98,  48, 45,  99,  99,  57, 101,  57,
   49, 101,  52,  48, 57,  57, 55,  34,  44,  34, 116, 104,
  105, 100,  34,  58, 34, 100, 49,  48,  99,  53,  54,  51,
   51,  45,  57, 101, 97,  50, 45,  52,  48,  55, 100,  45,
   57,  48,  98,  48, 45,  99, 99,  57, 101,  57,  49, 101,
   52,  48,  57,  57, 55,  34, 44,  34, 116, 121, 112,  34,
   58,  34,  97, 112,
  ... 274 more items
]


In [54]:
// generate the auth response and sign it (returns the request, response, and JWZ token)
let handleAuthResult = await authHandler.handleAuthorizationRequestForGenesisDID(userDID, authRawRequest)
handleAuthResult

{
  authRequest: {
    id: 'd10c5633-9ea2-407d-90b0-cc9e91e40997',
    thid: 'd10c5633-9ea2-407d-90b0-cc9e91e40997',
    typ: 'application/iden3comm-plain-json',
    type: 'https://iden3-communication.io/authorization/1.0/request',
    from: 'did:polygonid:polygon:mumbai:2qDrzvE2A813Fyg2sPE7JEA6f6yDsUJfzHXAhz9hZV',
    body: {
      callbackUrl: 'http://testcallback.com?sessionId=12345',
      reason: 'sign in',
      scope: []
    }
  },
  authResponse: {
    id: '499826dc-9903-47c0-964e-c7a5b5c25912',
    typ: 'application/iden3-zkp-json',
    type: 'https://iden3-communication.io/authorization/1.0/response',
    thid: 'd10c5633-9ea2-407d-90b0-cc9e91e40997',
    body: { did_doc: undefined, message: undefined, scope: [] },
    from: 'did:iden3:polygon:mumbai:wuyewEJyZZYbXvHtoc844eQHUeK17CkeWDWj3gwfY',
    to: 'did:polygonid:polygon:mumbai:2qDrzvE2A813Fyg2sPE7JEA6f6yDsUJfzHXAhz9hZV'
  },
  token: 'eyJhbGciOiJncm90aDE2IiwiY2lyY3VpdElkIjoiYXV0aFYyIiwiY3JpdCI6WyJjaXJjdWl0SWQiXSwidHlwIjoiY

In [55]:
import { Token } from "@iden3/js-jwz";

// The JWZ (JSON Web Zero-knowledge) token is a kind of JWT which encodes a ZKP signature of a message.
// For details see https://wiki.polygon.technology/docs/polygonid/verifier/verification-library/jwz/
let token = await Token.parse(handleAuthResult.token)
token

Token {
  method: ProvingMethodGroth16AuthV2 {
    methodAlg: ProvingMethodAlg { alg: 'groth16', circuitId: 'authV2' }
  },
  inputsPreparer: undefined,
  zkProof: {
    proof: {
      pi_a: [Array],
      pi_b: [Array],
      pi_c: [Array],
      protocol: 'groth16',
      curve: 'bn128'
    },
    pub_signals: [
      '21030615527935288911659687941340952125896174970295004076358028546924220929',
      '7461708270989948209333835316927808797544598293266956134503124760522018747999',
      '18706393090173903250541250945795870324119581947702997136924921910658951744668'
    ]
  },
  alg: 'groth16',
  circuitId: 'authV2',
  raw: {
    header: {
      alg: 'groth16',
      crit: [Array],
      circuitId: 'authV2',
      typ: 'application/iden3-zkp-json'
    },
    payload: Uint8Array(368) [
      123,  34, 105, 100,  34,  58,  34,  52,  57,  57,  56,  50,
       54, 100,  99,  45,  57,  57,  48,  51,  45,  52,  55,  99,
       48,  45,  57,  54,  52, 101,  45,  99,  55,  97,  53,  98,
       

In [56]:
// the message hash is one of the ZKP's public signals needed to verify the proof
core.fromBigEndian(await token.getMessageHash())

7461708270989948209333835316927808797544598293266956134503124760522018747999n


In [57]:
// the token's payload is the auth response (excluding undefined values)
JSON.parse(token.getPayload())

{
  id: '499826dc-9903-47c0-964e-c7a5b5c25912',
  typ: 'application/iden3-zkp-json',
  type: 'https://iden3-communication.io/authorization/1.0/response',
  thid: 'd10c5633-9ea2-407d-90b0-cc9e91e40997',
  body: { scope: [] },
  from: 'did:iden3:polygon:mumbai:wuyewEJyZZYbXvHtoc844eQHUeK17CkeWDWj3gwfY',
  to: 'did:polygonid:polygon:mumbai:2qDrzvE2A813Fyg2sPE7JEA6f6yDsUJfzHXAhz9hZV'
}


## Issuer issues a credential

In [58]:
import { CredentialRequest } from "@0xpolygonid/js-sdk"

// request to create a core claim for a W3C credential
const credentialRequest: CredentialRequest = {
    credentialSchema: "https://raw.githubusercontent.com/iden3/claim-schema-vocab/main/schemas/json/KYCAgeCredential-v3.json",
    type: "KYCAgeCredential",
    credentialSubject: {
      id: userDID.toString(),
      birthday: 19960424,
      documentType: 99,
    },
    expiration: 12345654321,
}

// issuer issues the credential
const credential = await identityWallet.issueCredential(
    issuerDID,
    credentialRequest,
    "http://wallet.com/", // host url that will a prefix of credential identifier
    {
      withRHS: rhsUrl,
    }
)

credential

W3CCredential {
  id: 'http://wallet.com/bd6b5873-1b05-49db-ae14-9eaaf01d8692',
  '@context': [
    'https://www.w3.org/2018/credentials/v1',
    'https://schema.iden3.io/core/jsonld/iden3proofs.jsonld',
    'https://raw.githubusercontent.com/iden3/claim-schema-vocab/main/schemas/json-ld/kyc-v3.json-ld'
  ],
  type: [ 'VerifiableCredential', 'KYCAgeCredential' ],
  expirationDate: '2361-03-21T12:25:21.000Z',
  issuanceDate: '2023-03-22T19:29:46.578Z',
  credentialSubject: {
    id: 'did:iden3:polygon:mumbai:wuyewEJyZZYbXvHtoc844eQHUeK17CkeWDWj3gwfY',
    birthday: 19960424,
    documentType: 99,
    type: 'KYCAgeCredential'
  },
  issuer: 'did:polygonid:polygon:mumbai:2qDrzvE2A813Fyg2sPE7JEA6f6yDsUJfzHXAhz9hZV',
  credentialSchema: {
    id: 'https://raw.githubusercontent.com/iden3/claim-schema-vocab/main/schemas/json/KYCAgeCredential-v3.json',
    type: 'JsonSchemaValidator2018'
  },
  credentialStatus: {
    id: 'https://rhs-staging.polygonid.me',
    revocationNonce: 4477,
    type:

In [59]:
// the proof is of type BJJ Signature and has the issuer's signature (BJJ signature on claim hash)
credential.proof[0]

{
  type: 'BJJSignature2021',
  issuerData: IssuerData {
    id: 'did:polygonid:polygon:mumbai:2qDrzvE2A813Fyg2sPE7JEA6f6yDsUJfzHXAhz9hZV',
    state: {
      rootOfRoots: '0000000000000000000000000000000000000000000000000000000000000000',
      revocationTreeRoot: '0000000000000000000000000000000000000000000000000000000000000000',
      claimsTreeRoot: '8b0a842a3be8aa86c4c4851636940e3fe20f8edcc8f9b6bd460d6c0f29c31b2a',
      value: '2474a161301ebc4b83c7214ca7e64a708228dbcd78f6a50826dc681627ad3025'
    },
    authCoreClaim: 'cca3371a6cb1b715004407e325bd993c000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000442c565c84d08f4eb369aab37485d1db0e614cd25224205228c1e8c419c40a1b553a0e3308c707d5d137b0e2adbb2f816bca173b8323240b4cd91032fcdeb71d000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000

In [60]:
credential.issuer == issuerDID.toString()

true


In [61]:
credential.credentialSubject.id == userDID.toString()

true


In [62]:
// for a Merklized W3C claim, index data slot A holds the Merkle tree hash
let claim = new core.Claim().fromHex(credential.proof[0].coreClaim)
let desc = describeClaim(claim)
desc

{
  schemaHash: 74977327600848231385663280181476307657n,
  id: 21030615527935288911659687941340952125896174970295004076358028546924220929n,
  indexData: [
    18928213416762675327354894866282314398660974085736279840820287826585392716665n,
    0n
  ],
  valueData: [ 0n, 0n ],
  expirationDate: 2361-03-21T12:25:21.000Z,
  merklizedRoot: 18928213416762675327354894866282314398660974085736279840820287826585392716665n,
  revocationNonce: 4477n,
  version: 0,
  flags: {
    subject: 'OtherIdenIndex',
    idPosition: 'Index',
    expiration: true,
    merklized: true,
    updatable: false
  }
}


In [63]:
// claim subject ID is the user / holder
userDID.toString()

did:iden3:polygon:mumbai:wuyewEJyZZYbXvHtoc844eQHUeK17CkeWDWj3gwfY


In [64]:
// save it (unlike auth claims, generic claims are not saved automatically)
await dataStorage.credential.saveCredential(credential);

## Query credential wallet

In [65]:
// summarize the stored credentials
let credentials = await dataStorage.credential.listCredentials()
console.table(
    credentials.map(cred => 
        _.pick(cred, ["id", "type", "issuer", "credentialSubject.id"])))

┌─────────┬──────────────────────────────────────────────────────────┬─────────────────────────────────────────────────┬───────────────────────────────────────────────────────────────────────────┬──────────────────────────────────────────────────────────────────────────────┐
│ (index) │                            id                            │                      type                       │                                  issuer                                   │                              credentialSubject                               │
├─────────┼──────────────────────────────────────────────────────────┼─────────────────────────────────────────────────┼───────────────────────────────────────────────────────────────────────────┼──────────────────────────────────────────────────────────────────────────────┤
│    0    │ 'http://wallet.com/8f6bd919-f88f-4885-b57a-6f6bfedc3930' │ [ 'VerifiableCredential', 'AuthBJJCredential' ] │ 'did:polygonid:polygon:mumbai:2qDrzvE2A813Fyg2sPE7J

In [66]:
// find credential by its id
await dataStorage.credential.findCredentialById(credential.id)

W3CCredential {
  id: 'http://wallet.com/bd6b5873-1b05-49db-ae14-9eaaf01d8692',
  '@context': [
    'https://www.w3.org/2018/credentials/v1',
    'https://schema.iden3.io/core/jsonld/iden3proofs.jsonld',
    'https://raw.githubusercontent.com/iden3/claim-schema-vocab/main/schemas/json-ld/kyc-v3.json-ld'
  ],
  type: [ 'VerifiableCredential', 'KYCAgeCredential' ],
  expirationDate: '2361-03-21T12:25:21.000Z',
  issuanceDate: '2023-03-22T19:29:46.578Z',
  credentialSubject: {
    id: 'did:iden3:polygon:mumbai:wuyewEJyZZYbXvHtoc844eQHUeK17CkeWDWj3gwfY',
    birthday: 19960424,
    documentType: 99,
    type: 'KYCAgeCredential'
  },
  issuer: 'did:polygonid:polygon:mumbai:2qDrzvE2A813Fyg2sPE7JEA6f6yDsUJfzHXAhz9hZV',
  credentialSchema: {
    id: 'https://raw.githubusercontent.com/iden3/claim-schema-vocab/main/schemas/json/KYCAgeCredential-v3.json',
    type: 'JsonSchemaValidator2018'
  },
  credentialStatus: {
    id: 'https://rhs-staging.polygonid.me',
    revocationNonce: 4477,
    type:

## Present auth challenge (request for proof)

Prereqs:
- User has a KYC Age credential issued to them in their Polygon ID wallet mobile app.
  - Use the issuer at https://issuer-demo.polygonid.me/ (via https://0xpolygonid.github.io/tutorials/issuer/demo-issuer/)
  - Set `userDID` to match be the user's DID from their Polygon ID (tap the person icon).

The request below states:
- Prove that you're an adult, with birthday prior to Jan 1, 2002,
  - as stated by a KYC age credential that you hold,
  - that was issued by any issuer.
- If so, invoke the `submitZKPResponse` function on the given contract (from the [IZKPVerifier](https://github.com/0xPolygonID/contracts/blob/main/contracts/interfaces/IZKPVerifier.sol#L8) interface),
  - including any public inputs needed to verify the proof.

The public inputs include the date to compare against (Jan 1, 2021) but not the user's birthday (considered PII).

The contract in this case is the [ERC20Verifier](https://github.com/0xPolygonID/tutorial-examples/blob/main/on-chain-verification/contracts/ERC20Verifier.sol) example contract, an ERC-20 token which extends [ZKPVerifier](https://github.com/0xPolygonID/contracts/blob/main/contracts/verifiers/ZKPVerifier.sol). Users who [submit a valid proof](https://github.com/0xPolygonID/contracts/blob/main/contracts/verifiers/ZKPVerifier.sol#L21) are [recorded and airdropped some tokens](https://github.com/0xPolygonID/tutorial-examples/blob/main/on-chain-verification/contracts/ERC20Verifier.sol#L38).

In [67]:
// ERC20zkAirdrop token at https://mumbai.polygonscan.com/address/0x0ee55ba4881477f3686d671257033524d550f70a
const tokenAddress = "0x0ee55ba4881477F3686d671257033524D550F70A"

In [68]:
import { AuthorizationRequestMessageBody } from "@0xpolygonid/js-sdk"

const proofReqSig: ZeroKnowledgeProofRequest = {
    id: 1,
    circuitId: CircuitId.AtomicQuerySigV2,
    // circuitId: "credentialAtomicQuerySigV2OnChain",
    // optional: false,
    query: {
      allowedIssuers: ["*"],
      context: "https://raw.githubusercontent.com/iden3/claim-schema-vocab/main/schemas/json-ld/kyc-v3.json-ld",
      credentialSubject: {
        birthday: {
          $lt: 20020101,
        },
      },
      type: credentialRequest.type,
    },
}

const threadId = "fe6354fe-3db2-48c2-a779-e39c2dda8d90"
let authRequest: AuthorizationRequestMessage = {
    id: threadId,
    thid: threadId,
    typ: PROTOCOL_CONSTANTS.MediaType.PlainMessage,
    from: issuerDID.toString(),
    type: PROTOCOL_CONSTANTS.PROTOCOL_MESSAGE_TYPE.AUTHORIZATION_REQUEST_MESSAGE_TYPE,
    // type: "https://iden3-communication.io/proofs/1.0/contract-invoke-request",
    body: {
      callbackUrl: "http://testcallback.com?sessionId=${sessionId}",
      // message: "message to sign",
      reason: "verify age",
      // transaction_data: {
      //   contract_address: tokenAddress,
      //   method_id: "b68967e2",  // 4-byte signature for `submitZKPResponse` function
      //   chain_id: 80001,
      //   network: "polygon-mumbai"
      // },
      scope: [proofReqSig],
    } as any as AuthorizationRequestMessageBody
}

JSON.stringify(authRequest, null, 2)

{
  "id": "fe6354fe-3db2-48c2-a779-e39c2dda8d90",
  "thid": "fe6354fe-3db2-48c2-a779-e39c2dda8d90",
  "typ": "application/iden3comm-plain-json",
  "from": "did:polygonid:polygon:mumbai:2qDrzvE2A813Fyg2sPE7JEA6f6yDsUJfzHXAhz9hZV",
  "type": "https://iden3-communication.io/authorization/1.0/request",
  "body": {
    "callbackUrl": "http://testcallback.com?sessionId=${sessionId}",
    "reason": "verify age",
    "scope": [
      {
        "id": 1,
        "circuitId": "credentialAtomicQuerySigV2",
        "query": {
          "allowedIssuers": [
            "*"
          ],
          "context": "https://raw.githubusercontent.com/iden3/claim-schema-vocab/main/schemas/json-ld/kyc-v3.json-ld",
          "credentialSubject": {
            "birthday": {
              "$lt": 20020101
            }
          },
          "type": "KYCAgeCredential"
        }
      }
    ]
  }
}


In [69]:
// present auth request as QR code so it can be scanned in Polygon ID app
let svg = qrSvg.QR(JSON.stringify(authRequest), "H");
tslab.display.html(`<div style="width: 300px; height: 300px;">${svg}</div>`)

## Respond to challenge

Using the Polygon ID mobile app, you can issue yourself a KYC Age credential using the [demo issuer](https://issuer-demo.polygonid.me/). See instructions [here](https://0xpolygonid.github.io/tutorials/issuer/demo-issuer/).

You can then scan the QR code above to respond to the challenge. 

Steps:
- launch the MetaMask mobile app and select the Mumbai testnet
  - add the network if needed via button at bottom https://mumbai.polygonscan.com
  - select the account / address you want to use to receive the airdrop
- launch the Polygon ID mobile app
- click Connect and scan the QR code above
- follow the prompts to generate proof
- if successful, it will open MetaMask and prompt to confirm transaction
- click Submit, which queues the transaction
- it switches back to Polygon ID app
- wait for transaction to complete, and check it in MetaMask
- you can navigate from there to view it in the blockchain explorer

In [70]:
authRequest.body.scope[0].query

{
  allowedIssuers: [ '*' ],
  context: 'https://raw.githubusercontent.com/iden3/claim-schema-vocab/main/schemas/json-ld/kyc-v3.json-ld',
  credentialSubject: { birthday: { '$lt': 20020101 } },
  type: 'KYCAgeCredential'
}


In [71]:
// find credentials in wallet matching the query
let matchingCredentials = await dataStorage.credential.findCredentialsByQuery(authRequest.body.scope[0].query)
matchingCredentials

[
  W3CCredential {
    id: 'http://wallet.com/bd6b5873-1b05-49db-ae14-9eaaf01d8692',
    '@context': [
      'https://www.w3.org/2018/credentials/v1',
      'https://schema.iden3.io/core/jsonld/iden3proofs.jsonld',
      'https://raw.githubusercontent.com/iden3/claim-schema-vocab/main/schemas/json-ld/kyc-v3.json-ld'
    ],
    type: [ 'VerifiableCredential', 'KYCAgeCredential' ],
    expirationDate: '2361-03-21T12:25:21.000Z',
    issuanceDate: '2023-03-22T19:29:46.578Z',
    credentialSubject: {
      id: 'did:iden3:polygon:mumbai:wuyewEJyZZYbXvHtoc844eQHUeK17CkeWDWj3gwfY',
      birthday: 19960424,
      documentType: 99,
      type: 'KYCAgeCredential'
    },
    issuer: 'did:polygonid:polygon:mumbai:2qDrzvE2A813Fyg2sPE7JEA6f6yDsUJfzHXAhz9hZV',
    credentialSchema: {
      id: 'https://raw.githubusercontent.com/iden3/claim-schema-vocab/main/schemas/json/KYCAgeCredential-v3.json',
      type: 'JsonSchemaValidator2018'
    },
    credentialStatus: {
      id: 'https://rhs-staging.pol

In [72]:
// select credentials matching the intended subject
if (authRequest.to) {
    matchingCredentials = matchingCredentials.filter(cred => cred.credentialSubject.id === authRequest.to)
}
matchingCredentials

[
  W3CCredential {
    id: 'http://wallet.com/bd6b5873-1b05-49db-ae14-9eaaf01d8692',
    '@context': [
      'https://www.w3.org/2018/credentials/v1',
      'https://schema.iden3.io/core/jsonld/iden3proofs.jsonld',
      'https://raw.githubusercontent.com/iden3/claim-schema-vocab/main/schemas/json-ld/kyc-v3.json-ld'
    ],
    type: [ 'VerifiableCredential', 'KYCAgeCredential' ],
    expirationDate: '2361-03-21T12:25:21.000Z',
    issuanceDate: '2023-03-22T19:29:46.578Z',
    credentialSubject: {
      id: 'did:iden3:polygon:mumbai:wuyewEJyZZYbXvHtoc844eQHUeK17CkeWDWj3gwfY',
      birthday: 19960424,
      documentType: 99,
      type: 'KYCAgeCredential'
    },
    issuer: 'did:polygonid:polygon:mumbai:2qDrzvE2A813Fyg2sPE7JEA6f6yDsUJfzHXAhz9hZV',
    credentialSchema: {
      id: 'https://raw.githubusercontent.com/iden3/claim-schema-vocab/main/schemas/json/KYCAgeCredential-v3.json',
      type: 'JsonSchemaValidator2018'
    },
    credentialStatus: {
      id: 'https://rhs-staging.pol

In [73]:
// scanning the QR code yields the auth request as bytes
let authRawRequest = new TextEncoder().encode(JSON.stringify(authRequest))
authRawRequest

Uint8Array(649) [
  123,  34, 105, 100,  34,  58,  34, 102, 101,  54,  51,  53,
   52, 102, 101,  45,  51, 100,  98,  50,  45,  52,  56,  99,
   50,  45,  97,  55,  55,  57,  45, 101,  51,  57,  99,  50,
  100, 100,  97,  56, 100,  57,  48,  34,  44,  34, 116, 104,
  105, 100,  34,  58,  34, 102, 101,  54,  51,  53,  52, 102,
  101,  45,  51, 100,  98,  50,  45,  52,  56,  99,  50,  45,
   97,  55,  55,  57,  45, 101,  51,  57,  99,  50, 100, 100,
   97,  56, 100,  57,  48,  34,  44,  34, 116, 121, 112,  34,
   58,  34,  97, 112,
  ... 549 more items
]


In [74]:
// generate the auth response and sign it (returns auth request, response, and JWZ token)
let handleAuthResult = await authHandler.handleAuthorizationRequestForGenesisDID(userDID, authRawRequest)
handleAuthResult

{
  authRequest: {
    id: 'fe6354fe-3db2-48c2-a779-e39c2dda8d90',
    thid: 'fe6354fe-3db2-48c2-a779-e39c2dda8d90',
    typ: 'application/iden3comm-plain-json',
    from: 'did:polygonid:polygon:mumbai:2qDrzvE2A813Fyg2sPE7JEA6f6yDsUJfzHXAhz9hZV',
    type: 'https://iden3-communication.io/authorization/1.0/request',
    body: {
      callbackUrl: 'http://testcallback.com?sessionId=${sessionId}',
      reason: 'verify age',
      scope: [Array]
    }
  },
  authResponse: {
    id: 'afb35d1c-100a-4f82-af16-e53b15ddc4fc',
    typ: 'application/iden3-zkp-json',
    type: 'https://iden3-communication.io/authorization/1.0/response',
    thid: 'fe6354fe-3db2-48c2-a779-e39c2dda8d90',
    body: { did_doc: undefined, message: undefined, scope: [Array] },
    from: 'did:iden3:polygon:mumbai:wuyewEJyZZYbXvHtoc844eQHUeK17CkeWDWj3gwfY',
    to: 'did:polygonid:polygon:mumbai:2qDrzvE2A813Fyg2sPE7JEA6f6yDsUJfzHXAhz9hZV'
  },
  token: 'eyJhbGciOiJncm90aDE2IiwiY2lyY3VpdElkIjoiYXV0aFYyIiwiY3JpdCI6WyJjaXJjd

In [75]:
handleAuthResult.authResponse.body.scope[0]

{
  id: 1,
  circuitId: 'credentialAtomicQuerySigV2',
  proof: {
    pi_a: [
      '6894795116143261445369993768054722363001349209927081733628945898540660572648',
      '20329002045415785402355888193456728909071711733204830464695451665032045002073',
      '1'
    ],
    pi_b: [ [Array], [Array], [Array] ],
    pi_c: [
      '7579302769507762719525830793173718520071431988682896020787721248557127672446',
      '17073197469701583603551079519562219816869590463284882098235326780203618778687',
      '1'
    ],
    protocol: 'groth16',
    curve: 'bn128'
  },
  pub_signals: [
    '1',
    '21030615527935288911659687941340952125896174970295004076358028546924220929',
    '16821579112606140904353438304433613438521059349428737335385863556345211483172',
    '1',
    '21734601890291053386314474656025329556666952536788676076983649605779329538',
    '1',
    '16821579112606140904353438304433613438521059349428737335385863556345211483172',
    '1679513408',
    '74977327600848231385663280181476307657',

In [76]:
let token = await Token.parse(handleAuthResult.token)
token

Token {
  method: ProvingMethodGroth16AuthV2 {
    methodAlg: ProvingMethodAlg { alg: 'groth16', circuitId: 'authV2' }
  },
  inputsPreparer: undefined,
  zkProof: {
    proof: {
      pi_a: [Array],
      pi_b: [Array],
      pi_c: [Array],
      protocol: 'groth16',
      curve: 'bn128'
    },
    pub_signals: [
      '21030615527935288911659687941340952125896174970295004076358028546924220929',
      '20988233609353342143308463065199345410357079333979199979041404343174977355395',
      '18706393090173903250541250945795870324119581947702997136924921910658951744668'
    ]
  },
  alg: 'groth16',
  circuitId: 'authV2',
  raw: {
    header: {
      alg: 'groth16',
      crit: [Array],
      circuitId: 'authV2',
      typ: 'application/iden3-zkp-json'
    },
    payload: Uint8Array(1899) [
      123,  34, 105, 100,  34,  58,  34,  97, 102,  98,  51,  53,
      100,  49,  99,  45,  49,  48,  48,  97,  45,  52, 102,  56,
       50,  45,  97, 102,  49,  54,  45, 101,  53,  51,  98,  49,
     

## Query the token contract

In [77]:
import { ethers } from "ethers"

// from https://github.com/0xPolygonID/contracts/blob/main/contracts/verifiers/ZKPVerifier.sol

const CircuitQuery = "struct (uint256 schema, uint256 claimPathKey, uint256 operator, uint256[] value, uint256 queryHash, string circuitId)"

const ICircuitValidator = "address"

const ERC20ABI = [ // a subset
    "function name() public view returns (string memory)",
    "function symbol() public view returns (string memory)",
    "function totalSupply() public view returns (uint256)",
    "function balanceOf(address account) public view returns (uint256)",
]

const ZKPVerifierABI = [
    "function getSupportedRequests() view returns (uint64[] arr)",
    `function getZKPRequest(uint64 requestId) view returns (${CircuitQuery})`,
    "function proofs(address account, uint64 requestId) view returns (bool)",
    `function requestQueries(uint64 requestId) view returns (${CircuitQuery})`,
    `function requestValidators(uint64 requestId) view returns (${ICircuitValidator})`,
    `function setZKPRequest(uint64 requestId, ${ICircuitValidator} validator, uint256 schema, uint256 claimPathKey, uint256 operator, uint256[] calldata value) returns (bool)`, // onlyOwner
    "function submitZKPResponse(uint64 requestId, uint256[] calldata inputs, uint256[2] calldata a, uint256[2][2] calldata b, uint256[2] calldata c) returns (bool)",
    "function submitZKPResponseRaw(uint64 requestId, uint256[] calldata inputs, uint256[2] calldata a, uint256[2][2] calldata b, uint256[2] calldata c, uint256 queryHash) returns (bool)",
]
    
const ethProvider = (dataStorage.states as EthStateStorage).provider
const tokenContract = new ethers.Contract(tokenAddress, [...ERC20ABI, ...ZKPVerifierABI], ethProvider)

In [78]:
// helper: recursively convert ethers.BigNumber to BigInt
function convertBigNums(o: any) {
    if (o instanceof ethers.BigNumber) {
        return o.toBigInt()
    }
    if (_.isArray(o)) {
        return o.map(convertBigNums)
    }
    if (typeof(o) === 'object') {
        return _.mapValues(o, convertBigNums)
    }
    return o;
}

In [79]:
// helper: convert an ethers function call result (an array with added named fields) to an object with just the named fields
function namedFields(o: any) {
    const keys: string[] = _.filter(_.keys(o), k => !k.match(/[0-9]+/));
    return _.fromPairs(keys.map(k => [k, o[k]]))
}

In [80]:
[await tokenContract.name(), await tokenContract.symbol()]

[ 'ERC20zkAirdrop', 'zkERC20' ]


In [81]:
// how many have been minted (5 are airdropped per valid claim)?
Number(await tokenContract.totalSupply()) / 1e18

5


In [82]:
// did we get the airdrop?
const walletAddress = "0x2aa06ccb57963f0c1ff0c29b009cd41509ee6818"
Number(await tokenContract.balanceOf(walletAddress)) / 1e18

5


In [83]:
// get the supported ZKP request IDs
convertBigNums(await tokenContract.getSupportedRequests())

[ 1n ]


In [84]:
// get the ZKP request (only the first value is used for operators EQ, LT, GT, NE)
convertBigNums(namedFields(await tokenContract.getZKPRequest(1)))

{
  schema: 74977327600848231385663280181476307657n,
  claimPathKey: 20376033832371109177683048456014525905119173674985843915445634726167450989630n,
  operator: 2n,
  value: [
    20020101n, 0n, 0n, 0n, 0n,
           0n, 0n, 0n, 0n, 0n,
           0n, 0n, 0n, 0n, 0n,
           0n, 0n, 0n, 0n, 0n,
           0n, 0n, 0n, 0n, 0n,
           0n, 0n, 0n, 0n, 0n,
           0n, 0n, 0n, 0n, 0n,
           0n, 0n, 0n, 0n, 0n,
           0n, 0n, 0n, 0n, 0n,
           0n, 0n, 0n, 0n, 0n,
           0n, 0n, 0n, 0n, 0n,
           0n, 0n, 0n, 0n, 0n,
           0n, 0n, 0n, 0n
  ],
  queryHash: 15045271939084694661437431358729281571840804299863053791890179002991342242959n,
  circuitId: 'credentialAtomicQuerySigV2OnChain'
}


In [85]:
import { Operators } from "@0xpolygonid/js-sdk"

namedFields(Operators)

{ NOOP: 0, EQ: 1, LT: 2, GT: 3, IN: 4, NIN: 5, NE: 6 }


In [86]:
await tokenContract.proofs(walletAddress, 1)

true


In [87]:
await tokenContract.requestValidators(1)

0xF2D4Eeb4d455fb673104902282Ce68B9ce4Ac450
